In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import os
import pickle
import numpy as np
from typing import Dict
import polars as pl

from src.common.AssetDataPolars import AssetDataPolars
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.featureAlchemy.FeatureFourierCoeff import FeatureFourierCoeff
from src.featureAlchemy.FeatureFourierCoeffTS import FeatureFourierCoeffTS


In [3]:
assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_debug")

# Convert to Polars for speedup
assetspl: Dict[str, AssetDataPolars] = {}
for ticker, asset in assets.items():
    assetspl[ticker] = AssetDataService.to_polars(asset)

In [4]:
asset = assetspl["AAPL"]
params = {
    'fouriercutoff': 5,
    'timesteps': 60,
    'lagList': [1, 2, 3, 5],
    'monthHorizonList': [1, 3, 6],
}

startDate = pd.Timestamp(2020, 1, 1, tz = "UTC").date()
endDate = pd.Timestamp(2021, 1, 1, tz = "UTC").date()
ffd = FeatureFourierCoeff(asset, startDate, endDate, params=params)

workdays = pd.date_range(start=dt.date(2020, 1, 1), end=dt.date(2020, 12, 31), freq='B')
dates = [date.date() for date in workdays]

In [5]:
featureNames = ffd.getFeatureNames()
features = ffd.apply(dates)

In [6]:
ffdts = FeatureFourierCoeffTS(assetspl[ticker], startDate, endDate, params=params)
featureNamesTimeseries = ffdts.getFeatureNames()
featuresTimeseries = ffdts.apply(dates)